FUSIÓN V4 inc Selenium+BS 12 atributos funcional

In [ ]:
!pip install selenium
import random
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd


# Lista de User-Agent para rotación
USER_AGENT = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/90.0.4430.93 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 Chrome/88.0.4324.96 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 Version/14.0.3 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 Chrome/89.0.4389.82 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:86.0) Gecko/20100101 Firefox/86.0"
]


# Definimos WebDriver
def crear_driver():
    """
    Iniciamos web driver.
    Args: lista de user agents.
    Returns: driver.
    """
    user_agent = random.choice(USER_AGENT)
    options = Options()
    options.add_argument(f'user-agent={user_agent}') # agregamos user agent
    print(f"User-Agent seleccionado: {user_agent}")
    options.add_argument('--headless') # operamos sin interfaz gráfica
    options.add_argument('--no-sandbox') # para evitar problemas de permisos en colab
    options.add_argument('--disable-dev-shm-usage') # deshabilitamos uso compartido de memoria
    driver = webdriver.Chrome(options=options)
    return driver
# https://stackoverflow.com/questions/69173469/meaning-of-selenium-chromeoptions
# https://medium.com/@moraneus/guide-to-using-python-selenium-873342d5fdad
# https://stackoverflow.com/questions/69173469/meaning-of-selenium-chromeoptions


def obtener_links_peliculas(driver):
    """
    Obtiene una lista con los links de las películas.
    Args: webdriver.
    Returns: cadena con los links de las películas encontradas.
    """

    try:
        mostrar_mas = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "load-more-bt"))
        )
        # Hacemos scroll
        driver.execute_script("arguments[0].scrollIntoView(true);", mostrar_mas)
        time.sleep(2)
        # Hacemos clic
        driver.execute_script("arguments[0].click();", mostrar_mas)
        print("Hacemos clic en botón mostrar más.")
        # Esperamos un momento para que se actualice la página
        time.sleep(5)
    except Exception as e:
        print("Error al hacer clic en el botón mostrar más:", e)


    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Buscamos elementos <a> que contengan /film en el atributo href (ver con inspector)
        # https://www.reddit.com/r/learnpython/comments/8173s3/beautifulsoup_parsing_and_regex_matching_the_end/
        # https://scrapeops.io/python-web-scraping-playbook/python-beautifulsoup-findall/#findall-using-regex
    objetos_peliculas = soup.find_all("a", href=re.compile("/film"))
    #print(objetos_peliculas)

    movie_links = []
    try:
      for objeto in objetos_peliculas:
          href = objeto["href"]
          #print(href)
          if "filmaffinity.com/es/film" in href and href not in movie_links:
            movie_links.append(href)
      print(f"Películas obtenidas en este paso: {len(movie_links)}")
    except:
      print("ERROR al hacer clic en el botón para mostrar más películas.")

    print(f"Se han encontrado {len(movie_links)} películas para analizar.")
    return movie_links


def obtener_premios_peliculas(driver):
    soup = BeautifulSoup(driver.page_source, "html.parser")
    premios_ganados = len(soup.find_all("span", class_="winner", string="ganadora"))
    premios_nominados = len(soup.find_all("span", class_="nom", string="nom."))

    print(f"Premios ganados: {premios_ganados}")
    print(f"Nominaciones: {premios_nominados}")

    return [premios_ganados,premios_nominados]


# Función para extraer información de una película
def get_movie_info(url):
    """
    Extrae los datos de las películas.
    """
    data = {}

    driver = crear_driver()
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    # ///// Título original /////
    try:
        # https://stackoverflow.com/questions/42894236/python-and-beautifulsoup-find-and-print-dd-list-items-by-finding-the-dt-text
        # https://www.educative.io/answers/how-to-use-gettext-in-beautiful-soup
        # https://python-forum.io/thread-16722.html
        dt = soup.find("dt", string=lambda text: text and "Título original" in text)
        dd = dt.find_next_sibling("dd")
        data["Título original"] = dd.get_text(strip=True)
        print(f"Título original: {data['Título original']}")
    except Exception as e:
        print("[ERROR] Extrayendo Título:", e)
        data["Título"] = None

    # ///// Año /////
    try:
        dt = soup.find("dt", string=lambda text: text and "Año" in text)
        dd = dt.find_next_sibling("dd")
        data["Año"] = dd.get_text(strip=True)
        print(f"Año: {data['Año']}")
    except Exception as e:
        print("[ERROR] Extrayendo Año:", e)
        data["Año"] = None

    # ///// Duración /////
    try:
        dt = soup.find("dt", string=lambda text: text and "Duración" in text)
        dd = dt.find_next_sibling("dd")
        data["Duración"] = dd.get_text(strip=True)
        print(f"Duración: {data['Duración']}")
    except Exception as e:
        print("[ERROR] Extrayendo Duración:", e)
        data["Duración"] = None

    # ///// País /////
    try:
        dt = soup.find("dt", string=lambda text: text and "País" in text)
        dd = dt.find_next_sibling("dd")
        data["País"] = dd.get_text(strip=True)
        print(f"País: {data['País']}")
    except Exception as e:
        print("[ERROR] Extrayendo País:", e)
        data["País"] = None

    # ///// Dirección /////
    try:
        dt = soup.find("dt", string=lambda text: text and "Dirección" in text)
        dd = dt.find_next_sibling("dd")
        data["Dirección"] = dd.get_text(strip=True)
        print(f"Dirección: {data['Dirección']}")
    except Exception as e:
        print("[ERROR] Extrayendo Dirección:", e)
        data["Dirección"] = None

    # ///// Género /////
    try:
        dt = soup.find("dt", string=lambda text: text and "Género" in text)
        dd = dt.find_next_sibling("dd")
        data["Género"] = dd.get_text(strip=True)
        print(f"Género: {data['Género']}")
    except Exception as e:
        print("[ERROR] Extrayendo Género:", e)
        data["Género"] = None

    # ///// Sinopsis /////
    try:
        dt = soup.find("dt", string=lambda text: text and "Sinopsis" in text)
        dd = dt.find_next_sibling("dd")
        data["Sinopsis"] = dd.get_text(strip=True)
        print(f"Sinopsis: {data['Sinopsis']}")
    except Exception as e:
        print("[ERROR] Extrayendo Sinopsis:", e)
        data["Sinopsis"] = None


    # ///// Valoración /////
    try:
        valoracion_div = soup.find('div', id='movie-rat-avg')
        data["Valoración"] = valoracion_div.text.strip()
        print(f"Valoración: {data['Valoración']}")
    except Exception as e:
        print("[ERROR] Extrayendo Valoración:", e)
        data["Valoración"] = None


    # ///// Número de votos /////

    try:
        votos_span = soup.find("span", itemprop="ratingCount")
        data["Número_Votos"] = int(votos_span.get("content"))
        print(f"Número de Votos: {data['Número_Votos']}")
    except Exception as e:
        print("[ERROR] Extrayendo Número de Votos:", e)
        data["Número_Votos"] = 0


    # ///// Número de Críticas /////
    try:
        valoracion_div = soup.find('div', id='movie-reviews-box')
        data["Número_Criticas"] = valoracion_div.text.strip()
        print(f"Número de Criticas: {data['Número_Criticas']}")
    except Exception as e:
        print("[ERROR] Extrayendo Número de Criticas:", e)
        data["Número_Criticas"] = None



    # ///// Número de Nominaciones /////
    try:
      #premios_dt = soup.find("dt", text="Premios")
      premios_dt = soup.find("dt", string="Premios")
      if premios_dt:
          print("Se encontró la sección de 'Premios'.")

          driver = crear_driver()

          # Abrimos la página principal de FilmAffinity
          print(f"Abrimos página: {url}")
          driver.get(url)
          time.sleep(random.uniform(1, 2))

          # Hacemos clic en el enlace "Mostrar todos"
          try:
              # Damos tiempo a que cargue la página
              mostrar_todos_link = WebDriverWait(driver, 5).until(
                  EC.element_to_be_clickable((By.CSS_SELECTOR, "i.fa-light.fa-chevron-right.strip-down"))
              )
              # Hacemos scroll por si el botón no estuviese a la vista
              driver.execute_script("arguments[0].scrollIntoView(true);", mostrar_todos_link)
              time.sleep(random.uniform(3,4))
              print("Esperando...")

              # Hacemos clic sobre el botón "Mostrar todos >"
              driver.execute_script("arguments[0].click();", mostrar_todos_link)
              print("Hacemos clic en 'Mostrar todos'", driver.current_url)
              time.sleep(random.uniform(4,5))
              print("Esperando...")
              movie_premios = obtener_premios_peliculas(driver)
              data["Número_Premios"] = movie_premios[0]
              data["Número_Nominaciones"] = movie_premios[1]
          except:
              print("[ERROR] Al hacer clic en 'Mostrar todos':")

          driver.quit()

    except Exception as e:
        print("[ERROR] Extrayendo Número de Nominaciones:", e)
        data["Número de Nominaciones"] = 0

    return data


def inicio_sesion(user,psw):
    try:
        ini_sesion_link = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.LINK_TEXT, "Iniciar sesión"))
        )
        # Hacemos scroll por si el botón no estuviese a la vista
        driver.execute_script("arguments[0].scrollIntoView(true);", ini_sesion_link)
        time.sleep(random.uniform(1, 2))
        print("Esperando...")

        driver.execute_script("arguments[0].click();", ini_sesion_link)
        print("Iniciamos sesión", driver.current_url)
    except Exception as e:
        print("[ERROR] Al hacer clic en iniciar sesión:")

    # Escribimos usuario
    user_input_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "user-input"))
    )
    user_input_field.send_keys(user)
    print(f"Se ha escrito {user} en el nombre de usuario.")

    # Escribimos contraseña
    user_input_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "psw-input"))
    )
    user_input_field.send_keys(psw)
    print(f"Se ha escrito {psw} en la contraseña.")

    # Hacemos clic en Iniciar sesión
    # https://uilicious.com/blog/how-to-click-a-button-using-selenium/#how-to-click-on-a-button-using-selenium
    try:
        boton_ini_sesion = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.login-btn"))
        )

        driver.execute_script("arguments[0].click();", boton_ini_sesion)
        print("Se ha hecho clic en Iniciar sesión.")
        time.sleep(random.uniform(4,5))
        return driver.current_url
    except Exception as e:
        print("[ERROR] al iniciar sesión")
        return e




## //////////////  INICIO  ////////////// ##

# Creamos driver para poder navegar por la web
driver = crear_driver()

# Abrimos la página principal de FilmAffinity
url_inicial = "https://www.filmaffinity.com/es/main.html"
print(f"Abrimos página: {url_inicial}")
driver.get(url_inicial)
time.sleep(random.uniform(1, 2))

# Iniciamos sesión
res = inicio_sesion(user="victorm8802@gmail.com",psw="Contraseña_film0")
print(res)
if res == url_inicial:
    print("Se ha iniciado sesión correctamente.")
else:
    print("No se ha podido iniciar sesión.")

# Hacemos clic en el enlace "Top FA"
try:
    # Damos tiempo a que cargue la página
    top_FA_link = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.LINK_TEXT, "Top FA"))
    )
    # Hacemos scroll por si el botón no estuviese a la vista
    driver.execute_script("arguments[0].scrollIntoView(true);", top_FA_link)
    time.sleep(random.uniform(1, 2))
    print("Esperando...")

    driver.execute_script("arguments[0].click();", top_FA_link)
    print("Hacemos clic en 'Top FA'", driver.current_url)
    time.sleep(random.uniform(1, 2))
    print("Esperando...")
except:
    print("[ERROR] Al hacer clic en 'Top FA':")


# Obtenemos links de las películas mostradas
movie_links = obtener_links_peliculas(driver)
driver.quit()

# Extraer datos de todas las películas
movies_data = []
for i, movie_url in enumerate(movie_links):
    # Limitamos a 50 películas para obtener una muestra.
    if i == 50: break
    print(f"\n[INFO] Extrayendo {i+1}/{len(movie_links)}: {movie_url}")
    movie_info = get_movie_info(movie_url)
    if movie_info:
        movies_data.append(movie_info)
    time.sleep(random.uniform(1, 2))

# Guardar en CSV
df = pd.DataFrame(movies_data)
df.to_csv("filmaffinity_movies.csv", index=False)

print("¡Dataset generado con éxito! 🚀")

# Guardamos en dataset
# movies_data
movies_data_df = pd.DataFrame(movies_data)
movies_data_df